In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import timeit
from sklearn.preprocessing import MinMaxScaler
from plotly.offline import init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

# Collaborative Filtering Model with Surprise Package

Melvin Ang, 5 March 2020, 290X 

In [2]:
df = pd.read_csv("df_Combined.csv")
df.head(10)

Name  16_Avenue_Tiled_Steps  California_Academy_of_Sciences  \
0        Swimmy128                   50.0                             NaN   
1        atriciaff                   50.0                             NaN   
2    Denise1236123                   50.0                             NaN   
3   reise-tanta-di                   50.0                             NaN   
4  ViorikaMontreal                   50.0                             NaN   
5        beltonlim                   50.0                             NaN   
6        justweems                   50.0                             NaN   
7           KissXX                   50.0                             NaN   
8         rosievil                   40.0                            50.0   
9    WaitForIttttt                   50.0                             NaN   

   Alcatraz_Island  Cable_Cars  Exploratorium  or5-Golden_Gate_Bridge  \
0              NaN         NaN            NaN                    40.0   
1              NaN        40.0            NaN                    50.0   
2             50.0         NaN            NaN                     NaN   
3              NaN         NaN            NaN                     NaN   
4              NaN         NaN            NaN                     NaN   
5              NaN        50.0           50.0                     NaN   
6              NaN         NaN            NaN                    50.0   
7              NaN         NaN            NaN                     NaN   
8              NaN        50.0            NaN                    50.0   
9             40.0        50.0            NaN                    50.0   

   Golden_Gate_Park  Lands_End  Oracle_Park  ...  Baker_Beach  \
0               NaN        NaN          NaN  ...          NaN   
1               NaN       50.0          NaN  ...          NaN   
2               NaN        NaN          NaN  ...          NaN   
3               NaN        NaN          NaN  ...          NaN   
4               NaN        NaN          NaN  ...          NaN   
5               NaN        NaN          NaN  ...          NaN   
6               NaN        NaN          NaN  ...          NaN   
7              50.0        NaN          NaN  ...          NaN   
8               NaN        NaN          NaN  ...          NaN   
9               NaN        NaN          NaN  ...         50.0   

   Angel_Island_State_Park  Mission_Dolores_Park  Fisherman_s_Wharf  \
0                      NaN                   NaN               30.0   
1                      NaN                   NaN                NaN   
2                      NaN                   NaN                NaN   
3                      NaN                   NaN                NaN   
4                      NaN                   NaN                NaN   
5                      NaN                   NaN               50.0   
6                      NaN                   NaN               30.0   
7                      NaN                   NaN                NaN   
8                      NaN                   NaN               50.0   
9                      NaN                   NaN               30.0   

   Japanese_Tea_Garden  Ghirardelli_Square  Chinatown  Union_Square  \
0                  NaN                 NaN        NaN           NaN   
1                  NaN                 NaN        NaN           NaN   
2                  NaN                 NaN        NaN           NaN   
3                  NaN                 NaN        NaN           NaN   
4                  NaN                 NaN        NaN           NaN   
5                  NaN                 NaN        NaN           NaN   
6                  NaN                40.0        NaN           NaN   
7                  NaN                 NaN        NaN           NaN   
8                  NaN                 NaN        NaN           NaN   
9                  NaN                 NaN        NaN           NaN   

   Painted_Ladies  Haight_Ashbury  
0             NaN             NaN  
1         

In [95]:
df_placenames = df.columns[1:].get_values()
df_placenames

array(['16_Avenue_Tiled_Steps', 'California_Academy_of_Sciences',
       'Alcatraz_Island', 'Cable_Cars', 'Exploratorium',
       'or5-Golden_Gate_Bridge', 'Golden_Gate_Park', 'Lands_End',
       'Oracle_Park', 'Palace_of_Fine_Arts_Theatre', 'Twin_Peaks',
       'Walt_Disney_Family_Museum', 'Ferry_Building_Marketplace',
       'Lombard_Street', 'San_Francisco_Museum_of_Modern_Art_SFMOMA',
       'Coit_Tower', 'Legion_of_Honor', 'San_Francisco_Bay', 'Pier_39',
       'or1395-Museum_of_Ice_Cream', 'San_Francisco_Botanical_Garden',
       'De_Young_Museum', 'Cable_Car_Museum', 'Presidio_of_San_Francisco',
       'Baker_Beach', 'Angel_Island_State_Park', 'Mission_Dolores_Park',
       'Fisherman_s_Wharf', 'Japanese_Tea_Garden', 'Ghirardelli_Square',
       'Chinatown', 'Union_Square', 'Painted_Ladies', 'Haight_Ashbury'],
      dtype=object)

In [4]:
#convert to Surprise package dataframe format
#df2.to_csv('df2.csv', index=False)
df_colnames = df.columns[1:].get_values()
df_colnames
df2 = pd.melt(df, id_vars=['Name'], value_vars=df_colnames, var_name='Place', value_name='Rating')
df2.shape

(1184900, 3)

In [8]:
#filter to reduce dataset 
min_place_ratings = 5
filter_place = df2['Place'].value_counts() > min_place_ratings
filter_place = filter_place[filter_place].index.tolist()
#df2.groupby('Place').count()['Rating'] 

min_user_ratings = 2
filter_users = df2.groupby('Name').count()['Rating'] > min_user_ratings #min 3 ratings 
filter_users = filter_users[filter_users].index.tolist()
#df2.groupby('Name').count().sort_values(by=['Rating'], ascending=False)['Rating']

df_new = df2[(df2['Place'].isin(filter_place)) & (df2['Name'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df2.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(1184900, 3)
The new data frame shape:	(160956, 3)


In [11]:
df_new = df_new.fillna(0)
df_new.shape

(160956, 3)

In [24]:
#divide all ratings by 10 so the scale becomes 1-5
df_new["Rating"] = df_new["Rating"].div(10)
df_new.describe()

Rating
count  160956.000000
mean        0.554344
std         1.492311
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         5.000000

In [29]:
df_new

Name                  Place  Rating
0                   Swimmy128  16_Avenue_Tiled_Steps     5.0
1                   atriciaff  16_Avenue_Tiled_Steps     5.0
2               Denise1236123  16_Avenue_Tiled_Steps     5.0
5                   beltonlim  16_Avenue_Tiled_Steps     5.0
6                   justweems  16_Avenue_Tiled_Steps     5.0
8                    rosievil  16_Avenue_Tiled_Steps     4.0
9               WaitForIttttt  16_Avenue_Tiled_Steps     5.0
10               WillieMacfan  16_Avenue_Tiled_Steps     5.0
11                    jnaggie  16_Avenue_Tiled_Steps     5.0
12                     Jrali1  16_Avenue_Tiled_Steps     5.0
14                  hollytrea  16_Avenue_Tiled_Steps     3.0
16                 conradm842  16_Avenue_Tiled_Steps     5.0
18                   SeaEmZed  16_Avenue_Tiled_Steps     5.0
19                  PirateJas  16_Avenue_Tiled_Steps     5.0
21                 timtex0527  16_Avenue_Tiled_Steps     3.0
22              LiveAGr8Story  16_Avenue_Tiled_Steps     4.0
23                   JennE402  16_Avenue_Tiled_Steps     4.0
24               markwN6292ZG  16_Avenue_Tiled_Steps     5.0
25                     tsb504  16_Avenue_Tiled_Steps     5.0
26                221natashat  16_Avenue_Tiled_Steps     3.0
28               Bobsilverado  16_Avenue_Tiled_Steps     5.0
29                    Subourd  16_Avenue_Tiled_Steps     5.0
33                    EmJayBV  16_Avenue_Tiled_Steps     5.0
35                  MalakoffR  16_Avenue_Tiled_Steps     5.0
36                    busconi  16_Avenue_Tiled_Steps     5.0
40             N3981JBmonicat  16_Avenue_Tiled_Steps     5.0
41                    ErnieKo  16_Avenue_Tiled_Steps     4.0
42             joanieatthefog  16_Avenue_Tiled_Steps     5.0
43                     bldit2  16_Avenue_Tiled_Steps     5.0
44                 jehenson81  16_Avenue_Tiled_Steps     4.0
...                       ...                    ...     ...
1180206                skytre         Haight_Ashbury     4.0
1180315               bbods13         Haight_Ashbury     0.0
1180564               L0ttieG         Haight_Ashbury     5.0
1180599              996juner         Haight_Ashbury     0.0
1180728                samitb         Haight_Ashbury     0.0
1180961         Culture467512         Haight_Ashbury     0.0
1181030       AprilScamFamMom         Haight_Ashbury     0.0
1181111          blackacre_11         Haight_Ashbury     4.0
1181313             Traflagar         Haight_Ashbury     0.0
1181363                 SVC17         Haight_Ashbury     0.0
1181382  happilyeverafter0406         Haight_Ashbury     2.0
1181385   FlorenceWManchester         Haight_Ashbury     3.0
1181420                razael         Haight_Ashbury     0.0
1181589               1mohman         Haight_Ashbury     0.0
1181801             MitchS616         Haight_Ashbury     0.0
1181814          IzzyEdmonton         Haight_Ashbury     0.0
1181825            tasangalli         Haight_Ashbury     0.0
1182039            950claudep         Haight_Ashbury     5.0
1182120            17shirleys         Haight_Ashbury     0.0
1182133       anacarolinaf467         Haight_Ashbury     3.0
1182288               ALF4121         Haight_Ashbury     5.0
1182332              jaben713         Haight_Ashbury     4.0
1182343              katjhart         Haight_Ashbury     0.0
1182553             975DEREKK         Haight_Ashbury     4.0
1182584             lindas286         Haight_Ashbury     4.0
1183325                283alc         Haight_Ashbury     5.0
1183652               Kerrylg         Haight_Ashbury     4.0
1183834            thorhansen         Haight_Ashbury     4.0
1183948             charmnuck         Haight_Ashbury     5.0
1183958               NedD767         Haight_Ashbury     3.0

[160956 rows x 3 columns]

In [30]:
#initiate Surprise
reader = Reader(rating_scale=(0, 5))
Surprise_data = Dataset.load_from_df(df_new[['Name', 'Place', 'Rating']], reader)

# Try Different Algorithms

In [145]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()

# Build an algorithm, and train it.
# Build an algorithm, and train it.
sim_options = {'name': 'cosine',
               'user_based': True #Similarity based on similar users
               }
algoKNNBasic = KNNBasic(sim_options=sim_options)
algoKNNBasic.fit(full_trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [146]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsKNNBasic = algo.fit(trainset).test(testset)
accuracy.rmse(predictionsKNNBasic)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.4251


1.4250829888161525

In [147]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoSVD = SVD()
algoSVD.fit(full_trainset)

In [149]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsSVD = algoSVD.fit(trainset).test(testset)
accuracy.rmse(predictionsSVD)

RMSE: 1.4486


1.4485835519932329

In [151]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
sim_options = {'name': 'cosine',
               'user_based': True #Similarity based on similar users
               }
algoKNNBaseline = KNNBaseline(sim_options=sim_options)
algoKNNBaseline.fit(full_trainset)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [152]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsKNNBaseline = algoKNNBaseline.fit(trainset).test(testset)
accuracy.rmse(predictionsKNNBaseline)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.4262


1.4261997669498734

In [154]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoBaselineOnly = BaselineOnly()
algoBaselineOnly.fit(full_trainset)

Estimating biases using als...


In [155]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsBaselineOnly = algoBaselineOnly.fit(trainset).test(testset)
accuracy.rmse(predictionsBaselineOnly)

Estimating biases using als...
RMSE: 1.4846


1.4845762391657442

In [157]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoCoClustering = CoClustering()
algoCoClustering.fit(full_trainset)

In [158]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsCoClustering = algoCoClustering.fit(trainset).test(testset)
accuracy.rmse(predictionsCoClustering)

RMSE: 1.5126


1.512581985164737

In [159]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoNormalPredictor = NormalPredictor()
algoNormalPredictor.fit(full_trainset)

In [160]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsNormalPredictor = algoNormalPredictor.fit(trainset).test(testset)
accuracy.rmse(predictionsNormalPredictor)

RMSE: 1.8636


1.8636400092448973

In [161]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
sim_options = {'name': 'cosine',
               'user_based': True #Similarity based on similar users
               }
algoKNNWithMeans = KNNWithMeans(sim_options=sim_options)
algoKNNWithMeans.fit(full_trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [162]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsKNNWithMeans = algoKNNWithMeans.fit(trainset).test(testset)
accuracy.rmse(predictionsKNNWithMeans)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.4360


1.4360302776389788

In [163]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
sim_options = {'name': 'cosine',
               'user_based': True #Similarity based on similar users
               }
algoKNNWithZScore = KNNWithZScore(sim_options=sim_options)
algoKNNWithZScore.fit(full_trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [164]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsKNNWithZScore = algoKNNWithZScore.fit(trainset).test(testset)
accuracy.rmse(predictionsKNNWithZScore)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.4394


1.4393954529215478

In [166]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoSVDpp = SVDpp()
algoSVDpp.fit(full_trainset)

In [167]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsSVDpp = algoSVDpp.fit(trainset).test(testset)
accuracy.rmse(predictionsSVDpp)

RMSE: 1.4821


1.4820760563768778

In [173]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoSlopeOne = SlopeOne()
algoSlopeOne.fit(full_trainset)

In [174]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsSlopeOne = algoSlopeOne.fit(trainset).test(testset)
accuracy.rmse(predictionsSlopeOne)

RMSE: 1.5009


1.5009175288343197

# Predicting

In [175]:
#predict 
uid = str("834tamih")  # raw user id (as in the ratings file). They are **strings**!
iid = str("Twin_Peaks")  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred=algoKNNBasic.predict(uid, iid, r_ui=5, verbose=True)

user: 834tamih   item: Twin_Peaks r_ui = 5.00   est = 0.27   {'actual_k': 40, 'was_impossible': False}


In [184]:
uid = str("834tamih")
user_list = []
place_list = []
rating_list = []
for i in range(len(df_placenames)):
    iid = df_placenames[i]
    #df_estimated_ratings = pd.DataFrame(algo.predict(uid, iid, verbose=True)[3])
    user_list.append(algoKNNBasic.predict(uid, iid, verbose=True)[0])
    place_list.append(algoKNNBasic.predict(uid, iid, verbose=True)[1])
    rating_list.append(algoKNNBasic.predict(uid, iid, verbose=True)[3])

user: 834tamih   item: 16_Avenue_Tiled_Steps r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: 16_Avenue_Tiled_Steps r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: 16_Avenue_Tiled_Steps r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: California_Academy_of_Sciences r_ui = None   est = 0.26   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: California_Academy_of_Sciences r_ui = None   est = 0.26   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: California_Academy_of_Sciences r_ui = None   est = 0.26   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Alcatraz_Island r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Alcatraz_Island r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Alcatraz_Island r_ui = None   est = 0.00   {'actua

user: 834tamih   item: Fisherman_s_Wharf r_ui = None   est = 0.12   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Fisherman_s_Wharf r_ui = None   est = 0.12   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Japanese_Tea_Garden r_ui = None   est = 0.50   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Japanese_Tea_Garden r_ui = None   est = 0.50   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Japanese_Tea_Garden r_ui = None   est = 0.50   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Ghirardelli_Square r_ui = None   est = 0.09   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Ghirardelli_Square r_ui = None   est = 0.09   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Ghirardelli_Square r_ui = None   est = 0.09   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Chinatown  r_ui = None   est = 0.20   {'actual_k': 40, 'was_impossible': False}
user: 83

In [185]:
#Print final result table
df_estimate_result = pd.DataFrame(user_list, columns =['Name'])
df_estimate_result["Place"] = pd.DataFrame(place_list)
df_estimate_result = df_estimate_result.merge(df_new[df_new["Name"] == uid], on='Place', how='outer').drop(['Name_y'], axis=1)
df_estimate_result["Estimated Rating"] = pd.DataFrame(rating_list)
df_estimate_result = df_estimate_result.rename(columns={"Name_x": "Name"})
df_estimate_result = df_estimate_result.rename(columns={"Rating": "Original Rating"})
df_estimate_result["Absolute Error"] = abs(df_estimate_result["Original Rating"] - df_estimate_result["Estimated Rating"])
df_estimate_result.sort_values(by=['Estimated Rating'],ascending=False)

Name                                      Place  Original Rating  \
23  834tamih                  Presidio_of_San_Francisco              5.0   
24  834tamih                                Baker_Beach              5.0   
25  834tamih                    Angel_Island_State_Park              5.0   
20  834tamih             San_Francisco_Botanical_Garden              4.0   
7   834tamih                                  Lands_End              0.0   
4   834tamih                              Exploratorium              0.0   
33  834tamih                             Haight_Ashbury              0.0   
16  834tamih                            Legion_of_Honor              0.0   
28  834tamih                        Japanese_Tea_Garden              0.0   
14  834tamih  San_Francisco_Museum_of_Modern_Art_SFMOMA              0.0   
21  834tamih                            De_Young_Museum              0.0   
9   834tamih                Palace_of_Fine_Arts_Theatre              0.0   
11  834tamih                  Walt_Disney_Family_Museum              0.0   
22  834tamih                           Cable_Car_Museum              0.0   
6   834tamih                           Golden_Gate_Park              0.0   
17  834tamih                          San_Francisco_Bay              0.0   
26  834tamih                       Mission_Dolores_Park              0.0   
31  834tamih                               Union_Square              0.0   
10  834tamih                                 Twin_Peaks              0.0   
1   834tamih             California_Academy_of_Sciences              0.0   
32  834tamih                             Painted_Ladies              0.0   
30  834tamih                                  Chinatown              0.0   
12  834tamih                 Ferry_Building_Marketplace              0.0   
27  834tamih                          Fisherman_s_Wharf              0.0   
29  834tamih                         Ghirardelli_Square              0.0   
3   834tamih                                 Cable_Cars              0.0   
18  834tamih                                    Pier_39              0.0   
8   834tamih                                Oracle_Park              0.0   
15  834tamih                                 Coit_Tower              0.0   
2   834tamih                            Alcatraz_Island              0.0   
5   834tamih                     or5-Golden_Gate_Bridge              0.0   
13  834tamih                             Lombard_Street              0.0   
19  834tamih                 or1395-Museum_of_Ice_Cream              0.0   
0   834tamih                      16_Avenue_Tiled_Steps              0.0   

    Estimated Rating  Absolute Error  
23          3.956940        1.043060  
24          2.966907        2.033093  
25          2.175897        2.824103  
20          1.953024        2.046976  
7           0.927657        0.927657  
4           0.543513        0.543513  
33          0.522273        0.522273  
16          0.517162        0.517162  
28          0.503824        0.503824  
14          0.496313        0.496313  
21          0.453123        0.453123  
9           0.434534        0.434534  
11          0.424371        0.424371  
22          0.349508        0.349508  
6           0.343141        0.343141  
17          0.340818        0.340818  
26          0.305083        0.305083  
31          0.274148        0.274148  
10          0.268072        0.268072  
1           0.255891        0.255891  
32          0.221275        0.221275  
30          0.204662        0.204662  
12          0.175969        0.175969  
27          0.116834        0.116834  
29          0.090613        0.090613  
3           0.089396        0.089396  
18          0.067047        0.067047  
8           0.066872        0.066872  
15          0.000000        0.000000  
2           0.000000        0.000000  
5           0.000000        0.000000  
13          0.000000        0.000000  
19          0.000000        0.000000  
0           0.000000 

In [71]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25)
#algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...


/Users/melvinang/anaconda3/lib/python3.7/site-packages/surprise/prediction_algorithms/algo_base.py:248: RuntimeWarning:

invalid value encountered in double_scalars



Done computing similarity matrix.
RMSE: 1.4203


1.4203341765796793

In [72]:
df_result = pd.DataFrame(predictions)
df_result

uid                                        iid  r_ui  \
0                kelsyf2016  San_Francisco_Museum_of_Modern_Art_SFMOMA   0.0   
1           TandCAdventures                              Exploratorium   0.0   
2              traveler1327                            Legion_of_Honor   0.0   
3                  anarayan             California_Academy_of_Sciences   0.0   
4                     CLT29                         Ghirardelli_Square   0.0   
5                 Alok_Bahl                              Exploratorium   5.0   
6              thakersuresh                              Exploratorium   0.0   
7          Whatsinaname7172             San_Francisco_Botanical_Garden   0.0   
8                LiannaS539                         Ghirardelli_Square   0.0   
9                   Pickids                                 Twin_Peaks   0.0   
10                  micchio                Palace_of_Fine_Arts_Theatre   0.0   
11           williewonker65             California_Academy_of_Sciences   0.0   
12           samuelmckinzie                Palace_of_Fine_Arts_Theatre   0.0   
13               MannyG2000                      16_Avenue_Tiled_Steps   0.0   
14          SallyCalifornia                              Exploratorium   0.0   
15               Mohit_Wad1                          Fisherman_s_Wharf   0.0   
16               Gooner10-8                             Lombard_Street   0.0   
17           nathandX5968YJ                     or5-Golden_Gate_Bridge   0.0   
18              ninasche117             San_Francisco_Botanical_Garden   0.0   
19                D_W_10017                           Cable_Car_Museum   0.0   
20           Traceycullen15                Palace_of_Fine_Arts_Theatre   0.0   
21             jammyjameela                       Mission_Dolores_Park   3.0   
22                   lotanp                     or5-Golden_Gate_Bridge   5.0   
23                    CANJG                                Baker_Beach   0.0   
24                Chelsea42  San_Francisco_Museum_of_Modern_Art_SFMOMA   5.0   
25            Travellerpics                                 Coit_Tower   0.0   
26                936briank                                 Twin_Peaks   0.0   
27             paulmH5689YV             San_Francisco_Botanical_Garden   0.0   
28               makeitso4u                             Lombard_Street   4.0   
29            Culture467512                                 Coit_Tower   0.0   
...                     ...                                        ...   ...   
40209              LAPBhona                           Golden_Gate_Park   0.0   
40210          johnsJ8383VA                    Angel_Island_State_Park   0.0   
40211        stevengU3104CH                                 Coit_Tower   0.0   
40212        Michelelabelle                                 Cable_Cars   0.0   
40213                toga1k                           Cable_Car_Museum   0.0   
40214               Kalmusc                    Angel_Island_State_Park   0.0   
40215           carlabqtpie                                Baker_Beach   0.0   
40216           373mauricew                         Ghirardelli_Square   4.0   
40217                jmdfaf                       Mission_Dolores_Park   0.0   
40218  TravellingBoneDoctor                                 Cable_Cars   4.0   
40219       anacarolinaf467                                  Lands_End   0.0   
40220        wantsadventure                              Exploratorium   5.0   
40221        GunterHeimlich  San_Francisco_Museum_of_Modern_Art_SFMOMA   0.0   
40222              891amana             California_Academy_of_Sciences   0.0   
40223          fernandowr10  San_Francisco_Museum_of_Modern_Art_SFMOMA   0.0   
40224             191semiha                                    Pier_39   0.0   
40225             memmottrw                    Angel_Island_State_Park   0.0   
40226                KH_RWC                      16_Avenue_Tiled_Steps   0.0   
40227         Luv2hikeju

In [80]:
#Check estimates for certain user
df_result[df_result["uid"] == "834tamih"]

uid                             iid  r_ui       est  \
3668   834tamih           16_Avenue_Tiled_Steps   0.0  0.674253   
8827   834tamih                 Legion_of_Honor   0.0  0.000000   
9764   834tamih               Fisherman_s_Wharf   0.0  0.050817   
10500  834tamih     Palace_of_Fine_Arts_Theatre   0.0  0.000000   
12277  834tamih                 De_Young_Museum   0.0  0.938937   
15972  834tamih       Presidio_of_San_Francisco   5.0  2.184342   
16222  834tamih                      Twin_Peaks   0.0  1.037484   
17659  834tamih                  Haight_Ashbury   0.0  0.974513   
26234  834tamih                     Oracle_Park   0.0  0.000000   
29313  834tamih                       Lands_End   0.0  4.500000   
29998  834tamih                 Alcatraz_Island   0.0  0.078566   
33488  834tamih                Golden_Gate_Park   0.0  1.057313   
34978  834tamih                      Cable_Cars   0.0  0.050157   
38689  834tamih  San_Francisco_Botanical_Garden   4.0  0.531329   

                                         details  
3668   {'actual_k': 19, 'was_impossible': False}  
8827    {'actual_k': 2, 'was_impossible': False}  
9764   {'actual_k': 35, 'was_impossible': False}  
10500   {'actual_k': 2, 'was_impossible': False}  
12277  {'actual_k': 21, 'was_impossible': False}  
15972  {'actual_k': 21, 'was_impossible': False}  
16222  {'actual_k': 22, 'was_impossible': False}  
17659  {'actual_k': 10, 'was_impossible': False}  
26234   {'actual_k': 7, 'was_impossible': False}  
29313   {'actual_k': 2, 'was_impossible': False}  
29998  {'actual_k': 25, 'was_impossible': False}  
33488  {'actual_k': 21, 'was_impossible': False}  
34978  {'actual_k': 24, 'was_impossible': False}  
38689  {'actual_k': 25, 'was_impossible': False}

In [77]:
#Check highest estimates
df_result.sort_values(by=['est'],ascending=False)

uid                                        iid  r_ui  \
21560            ConnorYM                                  Lands_End   0.0   
5958               K393YT                           Golden_Gate_Park   0.0   
23012            BethB246                     or5-Golden_Gate_Bridge   0.0   
24259           AzraelMel                           Golden_Gate_Park   0.0   
29313            834tamih                                  Lands_End   0.0   
18287        537beverleyg                     or5-Golden_Gate_Bridge   0.0   
9495             Laurie48                                  Lands_End   5.0   
4819     Adelaide_girl_SA                Palace_of_Fine_Arts_Theatre   5.0   
12086    Seasondtraveller                            De_Young_Museum   5.0   
560            joanneb648                     or5-Golden_Gate_Bridge   4.0   
21086  WednesdayNightGirl                                 Cable_Cars   0.0   
4425           raindrop26                            De_Young_Museum   5.0   
15289           37doulton                     or5-Golden_Gate_Bridge   5.0   
39835           Hison1961                     or5-Golden_Gate_Bridge   0.0   
25104           Trish1781                     or5-Golden_Gate_Bridge   3.0   
8000              esamlen                            De_Young_Museum   1.0   
23339               beim4                     or5-Golden_Gate_Bridge   4.0   
13933             Fazzers                     or5-Golden_Gate_Bridge   5.0   
26270          885jeannea                     or5-Golden_Gate_Bridge   0.0   
3093            128mariep                     or5-Golden_Gate_Bridge   5.0   
24004     Modestotrekkerl                            Legion_of_Honor   5.0   
7116   ClermontTravel1945                     or5-Golden_Gate_Bridge   0.0   
36980              rdpine                     or5-Golden_Gate_Bridge   5.0   
12797          stonerose5                                 Cable_Cars   0.0   
11803       chrisfJ3717RU                     or5-Golden_Gate_Bridge   4.0   
38938              pkcush             California_Academy_of_Sciences   5.0   
1351            tonytt_11                     or5-Golden_Gate_Bridge   0.0   
37583           beckyc447                     or5-Golden_Gate_Bridge   3.0   
16746          Sam_geddes                     or5-Golden_Gate_Bridge   4.0   
27753        steven010203                     or5-Golden_Gate_Bridge   5.0   
...                   ...                                        ...   ...   
12340           45Frank10                    Angel_Island_State_Park   0.0   
31111    markthesharkOhio                 or1395-Museum_of_Ice_Cream   0.0   
31145             tinkati                            De_Young_Museum   0.0   
12331     stephenbW6996VK                  Walt_Disney_Family_Museum   0.0   
12326        AKcabingirlA                       Mission_Dolores_Park   0.0   
31151             Pakiana                            Alcatraz_Island   0.0   
12322           helens353  San_Francisco_Museum_of_Modern_Art_SFMOMA   0.0   
12319          605martynh                            Legion_of_Honor   0.0   
12318               SS450                Palace_of_Fine_Arts_Theatre   0.0   
12370        Invisibl3Kid                      16_Avenue_Tiled_Steps   0.0   
31109              SeanJ5                  Walt_Disney_Family_Museum   0.0   
31052             SAB5859                           Cable_Car_Museum   0.0   
31067         glendale215                             Haight_Ashbury   0.0   
12443              Bunty8             San_Francisco_Botanical_Garden   0.0   
12440           Laura99IN                    Angel_Island_State_Park   0.0   
12439            Diane640                                  Lands_End   0.0   
31059            Yanzo_FG                                Baker_Beach   0.0   
31060           836eldent                    Angel_Island_State_Park   0.0   
12429          tksjourney                             Haight_Ashbury   0.0   
12427          diane33405                   

In [90]:
#Compile original user rating and estimated rating by model
dfresult_combined = df_new[df_new["Name"] == "834tamih"]
dfresult_combined = dfresult_combined.rename(columns={"Place": "iid"})
dfresult_combined = dfresult_combined.merge(df_result[df_result["uid"] == "834tamih"], on='iid', how='outer').drop(['uid', 'r_ui', 'details'], axis=1)
dfresult_combined

Name                                        iid  Rating       est
0   834tamih                      16_Avenue_Tiled_Steps     0.0  0.674253
1   834tamih             California_Academy_of_Sciences     0.0       NaN
2   834tamih                            Alcatraz_Island     0.0  0.078566
3   834tamih                                 Cable_Cars     0.0  0.050157
4   834tamih                              Exploratorium     0.0       NaN
5   834tamih                     or5-Golden_Gate_Bridge     0.0       NaN
6   834tamih                           Golden_Gate_Park     0.0  1.057313
7   834tamih                                  Lands_End     0.0  4.500000
8   834tamih                                Oracle_Park     0.0  0.000000
9   834tamih                Palace_of_Fine_Arts_Theatre     0.0  0.000000
10  834tamih                                 Twin_Peaks     0.0  1.037484
11  834tamih                  Walt_Disney_Family_Museum     0.0       NaN
12  834tamih                 Ferry_Building_Marketplace     0.0       NaN
13  834tamih                             Lombard_Street     0.0       NaN
14  834tamih  San_Francisco_Museum_of_Modern_Art_SFMOMA     0.0       NaN
15  834tamih                                 Coit_Tower     0.0       NaN
16  834tamih                            Legion_of_Honor     0.0  0.000000
17  834tamih                          San_Francisco_Bay     0.0       NaN
18  834tamih                                    Pier_39     0.0       NaN
19  834tamih                 or1395-Museum_of_Ice_Cream     0.0       NaN
20  834tamih             San_Francisco_Botanical_Garden     4.0  0.531329
21  834tamih                            De_Young_Museum     0.0  0.938937
22  834tamih                           Cable_Car_Museum     0.0       NaN
23  834tamih                  Presidio_of_San_Francisco     5.0  2.184342
24  834tamih                                Baker_Beach     5.0       NaN
25  834tamih                    Angel_Island_State_Park     5.0       NaN
26  834tamih                       Mission_Dolores_Park     0.0       NaN
27  834tamih                          Fisherman_s_Wharf     0.0  0.050817
28  834tamih                        Japanese_Tea_Garden     0.0       NaN
29  834tamih                         Ghirardelli_Square     0.0       NaN
30  834tamih                                  Chinatown     0.0       NaN
31  834tamih                               Union_Square     0.0       NaN
32  834tamih                             Painted_Ladies     0.0       NaN
33  834tamih                             Haight_Ashbury     0.0  0.974513

In [ ]:
predictions2 = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)